<h1><center>OPTUNA vs БАЙЕС</center></h1>

Вернёмся к анализу датасета «Космолайна» об оттоке клиентов. До этого вы применяли к нему методы Grid Search и Random Search для оптимизации гиперпараметров модели. Новая задача — использовать байесовский подход. Вы будете работать с алгоритмом TPE. 

Как и до этого, вам предстоит обучить новую версию модели — но в этот раз интегрировав процесс обучения с библиотекой optuna для оптимизации гиперпараметров. Так, вы не только автоматизируете поиск наилучших настроек для модели, но и сделаете его более точным и целенаправленным. 

Вы также потренируетесь проводить интеграцию с MLflow. Напомним, что это позволяет систематизировать и сохранять информацию обо всех экспериментах, проведённых в ходе подбора гиперпараметров. И к тому же поможет анализировать результаты.

Не забудем и про воспроизводимость. Для этого вы сохраните всю информацию о процессе подбора гиперпараметров в локальную базу данных. В реальных проектах это будет отличной страховкой, ведь вы сможете в любой момент восстановить условия эксперимента, повторить его и проверить результаты.

Так начнём же.

---

Импортируем библиотеки и настроим параметры:

In [1]:
import pandas as pd
import numpy as np
import os
import psycopg
import mlflow
from mlflow.tracking import MlflowClient
from catboost import CatBoostClassifier
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import ( OneHotEncoder, SplineTransformer, QuantileTransformer, StandardScaler,
                                    RobustScaler, PolynomialFeatures, KBinsDiscretizer )
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from autofeat import AutoFeatRegressor, AutoFeatClassifier

import catboost as cb
from catboost import CatBoostClassifier
from category_encoders import CatBoostEncoder

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import Lasso
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

import optuna
from optuna.integration.mlflow import MLflowCallback
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


TABLE_NAME = "clean_users_churn" # таблица с данными
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

EXPERIMENT_NAME = "model_bayesian_search" # ваш код здесь
RUN_NAME = "model_bayesian_search"
REGISTRY_MODEL_NAME = 'model_bayesian_search' # ваш код здесь
MLFLOW_PARENT_RUN_ID = 'model_bayesian_search'

STUDY_DB_NAME = "sqlite:///local.study.db"
STUDY_NAME = "model_bayesian_search"

experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)\
    if not mlflow.get_experiment_by_name(EXPERIMENT_NAME)\
    else mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

/home/mle-user/mle_projects/mle-mlflow/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Загрузим таблицу:

In [2]:
connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {"host": 'rc1b-uh7kdmcx67eomesf.mdb.yandexcloud.net', #os.getenv("DB_DESTINATION_HOST"),
                        "port": '6432', #os.getenv("DB_DESTINATION_PORT"),
                        "dbname": 'playground_mle_20250529_05fed48463', #os.getenv("DB_DESTINATION_NAME"),
                        "user": 'mle_20250529_05fed48463', #os.getenv("DB_DESTINATION_USER"),
                        "password": '0c567edd8ad8472e87d5c85cc4d664e4' } #os.getenv("DB_DESTINATION_PASSWORD")}
connection.update(postgres_credentials)

with psycopg.connect(**connection) as conn:
    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
        data = cur.fetchall()
        columns = [col[0] for col in cur.description]

df = pd.DataFrame(data, columns=columns)
df['target'] = (df['end_date'].notna()).astype(int)
df.head(2) 

,id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,...,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target
0,1,7590-VHVEG,2020-01-01,NaT,Month-to-month,Yes,Electronic check,29.85,29.85,DSL,...,No,No,No,No,Female,0,Yes,No,No,0
1,2,5575-GNVDE,2017-04-01,NaT,One year,No,Mailed check,56.95,1889.50,DSL,...,Yes,No,No,No,Male,0,No,No,No,0


Подготовим данные для обучения:

Выберите команду, которая выделит нечисловые колонки вашего датасета:

In [17]:
obj_df = df.select_dtypes(include="object")

**Задание 1**

После того как вы выделили категориальные колонки, закодируйте их для подачи в вашу модель:

In [18]:
# определение категориальных колонок, которые будут преобразованы
cat_columns = ["type", "payment_method", "internet_service", "gender"]

# создание объекта OneHotEncoder для преобразования категориальных переменных
# auto - автоматическое определение категорий
# ignore - игнорировать ошибки, если встречается неизвестная категория
# max_categories - максимальное количество уникальных категорий
# sparse_output - вывод в виде разреженной матрицы, если False, то в виде обычного массива
# drop="first" - удаляет первую категорию, чтобы избежать ловушки мультиколлинеарности
encoder_oh = OneHotEncoder(categories='auto', handle_unknown='ignore', max_categories=10, sparse_output=False, drop='first') # ваш код здесь #

# применение OneHotEncoder к данным. Преобразование категориальных данных в массив
encoded_features = encoder_oh.fit_transform(df[cat_columns].to_numpy()) # ваш код здесь #

# преобразование полученных признаков в DataFrame и установка названий колонок
# get_feature_names_out() - получение имён признаков после преобразования
encoded_df = pd.DataFrame(encoded_features, columns=encoder_oh.get_feature_names_out(cat_columns)) # ваш код здесь #

# конкатенация исходного DataFrame с новым DataFrame, содержащим закодированные категориальные признаки
# axis=1 означает конкатенацию по колонкам
obj_df = pd.concat([obj_df, encoded_df], axis=1)

obj_df.head(2)

,type,paperless_billing,payment_method,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,...,partner,dependents,multiple_lines,type_One year,type_Two year,payment_method_Credit card (automatic),payment_method_Electronic check,payment_method_Mailed check,internet_service_Fiber optic,gender_Male
3707,One year,No,Credit card (automatic),Fiber optic,No,No,No,No,No,No,...,No,No,No,0.0,0.0,0.0,1.0,0.0,1.0,1.0
1524,Two year,No,Bank transfer (automatic),Fiber optic,No,No,No,No,No,No,...,Yes,No,No,0.0,0.0,0.0,0.0,1.0,0.0,1.0


Сейчас поработайте с числовыми признаками: monthly_charges и total_charges. Из них можно сгенерировать довольно много признаков для вашей модели. 

**Задание 2**

Напишите код преобразования числовых признаков в списке num_columns, используя следующие энкодеры:
- SplineTransformer,
- QuantileTransformer,
- RobustScaler,
- PolynomialFeatures,
- KBinsDiscretizer.

In [19]:
num_columns = ["monthly_charges", "total_charges"]

n_knots = 3
degree_spline = 4
n_quantiles=100
degree = 3
n_bins = 5
encode = 'ordinal'
strategy = 'uniform'
subsample = None

# num_df = df.select_dtypes(include=['number'])

num_df = df[num_columns].copy()

# SplineTransformer
encoder_spl = SplineTransformer(n_knots=n_knots, degree=degree_spline) # ваш код здесь #
encoded_features = encoder_spl.fit_transform(df[num_columns].to_numpy()) # ваш код здесь #
encoded_df = pd.DataFrame( encoded_features, columns=encoder_spl.get_feature_names_out(num_columns) )
num_df = pd.concat([num_df, encoded_df], axis=1)

# QuantileTransformer
encoder_q = QuantileTransformer(n_quantiles=n_quantiles) #, output_distribution='normal') # ваш код здесь #
encoded_features = encoder_q.fit_transform(df[num_columns].to_numpy()) # ваш код здесь #
encoded_df = pd.DataFrame(encoded_features, columns=encoder_q.get_feature_names_out(num_columns)) # ваш код здесь #
encoded_df.columns = [col + f"_q_{n_quantiles}" for col in num_columns]
num_df = pd.concat([num_df, encoded_df], axis=1)

# RobustScaler
encoder_rb = RobustScaler() # ваш код здесь #
encoded_features = encoder_rb.fit_transform(df[num_columns].to_numpy()) # ваш код здесь #
encoded_df = pd.DataFrame(encoded_features, columns=encoder_rb.get_feature_names_out(num_columns)) # ваш код здесь #
encoded_df.columns = [col + f"_robust" for col in num_columns]
num_df = pd.concat([num_df, encoded_df], axis=1)

# PolynomialFeatures
encoder_pol = PolynomialFeatures(degree=degree) # ваш код здесь #
encoded_features = encoder_pol.fit_transform(df[num_columns].to_numpy()) # ваш код здесь #
encoded_df = pd.DataFrame(encoded_features, columns=encoder_pol.get_feature_names_out(num_columns)) # ваш код здесь #
# get all columns after the intercept and original features
encoded_df.columns = encoder_pol.get_feature_names_out(num_columns)
encoded_df = encoded_df.iloc[:, 1 + len(num_columns):]
encoded_df.columns = [f"{col}_poly" for col in encoded_df.columns]

# KBinsDiscretizer
encoder_kbd = KBinsDiscretizer(n_bins=n_bins, encode=encode, strategy=strategy, subsample=subsample) # ваш код здесь #
encoded_features = encoder_kbd.fit_transform(df[num_columns].to_numpy()) # ваш код здесь #
encoded_df = pd.DataFrame(encoded_features, columns=encoder_kbd.get_feature_names_out(num_columns)) # ваш код здесь #
encoded_df.columns = [col + f"_bin" for col in num_columns]
num_df = pd.concat([num_df, encoded_df], axis=1) # ваш код здесь #

num_df.head(2)

,monthly_charges,total_charges,monthly_charges_sp_0,monthly_charges_sp_1,monthly_charges_sp_2,monthly_charges_sp_3,monthly_charges_sp_4,monthly_charges_sp_5,total_charges_sp_0,total_charges_sp_1,total_charges_sp_2,total_charges_sp_3,total_charges_sp_4,total_charges_sp_5,monthly_charges_q_100,total_charges_q_100,monthly_charges_robust,total_charges_robust,monthly_charges_bin,total_charges_bin
3707,18.25,534.70,0.000000,0.026679,0.403382,0.507733,0.062201,0.000005,0.020923,0.374849,0.529355,0.074847,0.000026,0.0,0.528832,0.337356,0.062818,-0.208140,2.0,0.0
1524,18.40,1057.85,0.022032,0.380798,0.525064,0.072086,0.000020,0.000000,0.027673,0.407787,0.504158,0.060379,0.000004,0.0,0.216330,0.260664,-0.826790,-0.286007,0.0,0.0


Трансформируем данные:

In [ ]:
numeric_transformer = ColumnTransformer(transformers=[('spl', encoder_spl, num_columns),('q', encoder_q, num_columns), ('rb', encoder_rb, num_columns), ('pol', encoder_pol, num_columns), ('kbd', encoder_kbd, num_columns)])
categorical_transformer = Pipeline(steps=[('encoder', encoder_oh)] )
preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, num_columns), ('cat', categorical_transformer, cat_columns)], n_jobs=-1)
encoded_features = preprocessor.fit_transform(df) # ваш код здесь #
transformed_df = pd.DataFrame(encoded_features, columns=preprocessor.get_feature_names_out()) # ваш код здесь #

df = pd.concat([df, transformed_df], axis=1) # ваш код здесь #
df.head(2)

,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,online_security,online_backup,device_protection,tech_support,...,num__pol__total_charges^3,num__kbd__monthly_charges,num__kbd__total_charges,cat__type_One year,cat__type_Two year,cat__payment_method_Credit card (automatic),cat__payment_method_Electronic check,cat__payment_method_Mailed check,cat__internet_service_Fiber optic,cat__gender_Male
3707,One year,No,Credit card (automatic),18.25,534.70,Fiber optic,No,No,No,No,...,3.493599e+08,2.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
1524,Two year,No,Bank transfer (automatic),18.40,1057.85,Fiber optic,No,No,No,No,...,8.530021e+07,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


Выберем закодированные данные:

In [ ]:
df = df[['target', 'num__spl__monthly_charges_sp_0',
       'num__spl__monthly_charges_sp_1', 'num__spl__monthly_charges_sp_2',
       'num__spl__monthly_charges_sp_3', 'num__spl__monthly_charges_sp_4',
       'num__spl__monthly_charges_sp_5', 'num__spl__total_charges_sp_0',
       'num__spl__total_charges_sp_1', 'num__spl__total_charges_sp_2',
       'num__spl__total_charges_sp_3', 'num__spl__total_charges_sp_4',
       'num__spl__total_charges_sp_5', 'num__q__monthly_charges',
       'num__q__total_charges', 'num__rb__monthly_charges',
       'num__rb__total_charges', 'num__pol__1', 'num__pol__monthly_charges',
       'num__pol__total_charges', 'num__pol__monthly_charges^2',
       'num__pol__monthly_charges total_charges', 'num__pol__total_charges^2',
       'num__pol__monthly_charges^3',
       'num__pol__monthly_charges^2 total_charges',
       'num__pol__monthly_charges total_charges^2',
       'num__pol__total_charges^3', 'num__kbd__monthly_charges',
       'num__kbd__total_charges', 'cat__type_One year', 'cat__type_Two year',
       'cat__payment_method_Credit card (automatic)',
       'cat__payment_method_Electronic check',
       'cat__payment_method_Mailed check', 'cat__internet_service_Fiber optic',
       'cat__gender_Male']]

# Разделение данных на обучающую и валидационную выборки
X_train, X_val, y_train, y_val = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.2, random_state=42)